# Load Library

In [1]:
import pandas as pd
import numpy as np
import warnings
from sklearn.decomposition import TruncatedSVD
from scipy.sparse.linalg import svds

warnings.filterwarnings('ignore')

# SVD -> 피어슨 상관계수를 이용한 추천

In [2]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')

In [3]:
rating_movie = pd.merge(ratings, movies, on='movieId')

In [4]:
rating_movie

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,7,31,3.0,851868750,Dangerous Minds (1995),Drama
2,31,31,4.0,1273541953,Dangerous Minds (1995),Drama
3,32,31,4.0,834828440,Dangerous Minds (1995),Drama
4,36,31,3.0,847057202,Dangerous Minds (1995),Drama
...,...,...,...,...,...,...
99999,664,64997,2.5,1343761859,War of the Worlds (2005),Action|Sci-Fi
100000,664,72380,3.5,1344435977,"Box, The (2009)",Drama|Horror|Mystery|Sci-Fi|Thriller
100001,665,129,3.0,995232528,Pie in the Sky (1996),Comedy|Romance
100002,665,4736,1.0,1010197684,Summer Catch (2001),Comedy|Drama|Romance


In [8]:
user_movie_rating = pd.pivot_table(rating_movie, index='userId', columns='title', values='rating').fillna(0)

In [9]:
user_movie_rating

title,"""Great Performances"" Cats (1998)",$9.99 (2008),'Hellboy': The Seeds of Creation (2004),'Neath the Arizona Skies (1934),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...And God Spoke (1993),...And Justice for All (1979),1-900 (06) (1994),10 (1979),10 Attitudes (2001),10 Cloverfield Lane (2016),10 Items or Less (2006),10 Things I Hate About You (1999),10 Years (2011),"10,000 BC (2008)",100 Girls (2000),100 Rifles (1969),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),101 Reykjavik (101 Reykjavík) (2000),102 Dalmatians (2000),"10th Kingdom, The (2000)","11'09""01 - September 11 (2002)",11:14 (2003),12 Angry Men (1957),12 Angry Men (1997),12 Years a Slave (2013),127 Hours (2010),12:08 East of Bucharest (A fost sau n-a fost?) (2006),13 Assassins (Jûsan-nin no shikaku) (2010),13 Ghosts (1960),13 Going on 30 (2004),13 Tzameti (2005),"13th Warrior, The (1999)",...,Zach Galifianakis: Live at the Purple Onion (2006),Zack and Miri Make a Porno (2008),Zardoz (1974),Zathura (2005),Zatoichi on the Road (Zatôichi kenka-tabi) (Zatôichi 5) (1963),Zazie dans le métro (1960),"Zed & Two Noughts, A (1985)",Zeitgeist: The Movie (2007),Zelary (2003),Zelig (1983),Zenon: Girl of the 21st Century (1999),Zenon: The Zequel (2001),Zenon: Z3 (2004),Zero Dark Thirty (2012),Zero Effect (1998),"Zero Theorem, The (2013)",Zerophilia (2005),Zodiac (2007),Zombeavers (2014),Zombie (a.k.a. Zombie 2: The Dead Are Among Us) (Zombi 2) (1979),Zombie Holocaust (a.k.a. Doctor Butcher M.D.) (Zombi Holocaust) (1980),Zombieland (2009),Zoolander (2001),Zoolander 2 (2016),Zoom (2006),Zoot Suit (1981),Zootopia (2016),Zorba the Greek (Alexis Zorbas) (1964),Zorn's Lemma (1970),"Zorro, the Gay Blade (1981)",Zulu (1964),Zulu (2013),[REC] (2007),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931),İtirazım Var (2014)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [13]:
movie_user_rating = user_movie_rating.values.T

In [15]:
movie_user_rating.shape

(9064, 671)

In [17]:
# TruncatedSVD를 사용해 차원 축소
SVD = TruncatedSVD(n_components=12)
matrix = SVD.fit_transform(movie_user_rating)
matrix.shape

(9064, 12)

In [18]:
corr = np.corrcoef(matrix)

In [25]:
movie_title = user_movie_rating.columns
movie_corr_df = pd.DataFrame(corr, columns=movie_title, index=movie_title)

In [28]:
def movie_recommend_by_corr(title, num):
  return movie_corr_df[title].sort_values(ascending=False)[1:num+1]

In [31]:
movie_title = "Guardians of the Galaxy (2014)"
num = 10
movie_recommend_by_corr(movie_title, num)

title
Iron Man 2 (2010)                            0.993868
Edge of Tomorrow (2014)                      0.992542
Avengers, The (2012)                         0.984908
X-Men: Days of Future Past (2014)            0.970323
Indie Game: The Movie (2012)                 0.969312
The Hunger Games (2012)                      0.968075
Captain America: The First Avenger (2011)    0.965777
X-Men: First Class (2011)                    0.962607
Drillbit Taylor (2008)                       0.960245
Paperboy, The (2012)                         0.960245
Name: Guardians of the Galaxy (2014), dtype: float64

# Matrix Factorization을 이용한 사용자 개인 추천

In [39]:
# user-movie 데이터프레임을 matrix로 변경
user_movie_rating_matrix = user_movie_rating.values
print(user_movie_rating_matrix)
print(user_movie_rating_matrix.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(671, 9064)


In [41]:
# user별 영화 평균 평점 
user_rating_mean = np.mean(, axis=1)user_movie_rating_matrix

In [44]:
# user-movie matrix에서 유저별 평균 평점을 빼줌
user_movie_rating_matrix_scaled = user_movie_rating_matrix - user_rating_mean.reshape(-1,1)

In [45]:
user_movie_rating_matrix_scaled

array([[-0.00562665, -0.00562665, -0.00562665, ..., -0.00562665,
        -0.00562665, -0.00562665],
       [-0.02923654, -0.02923654, -0.02923654, ..., -0.02923654,
        -0.02923654, -0.02923654],
       [-0.02007944, -0.02007944, -0.02007944, ..., -0.02007944,
        -0.02007944, -0.02007944],
       ...,
       [-0.01368049, -0.01368049, -0.01368049, ..., -0.01368049,
        -0.01368049, -0.01368049],
       [-0.01301853, -0.01301853, -0.01301853, ..., -0.01301853,
        -0.01301853, -0.01301853],
       [-0.04970212, -0.04970212, -0.04970212, ..., -0.04970212,
        -0.04970212, -0.04970212]])

In [46]:
# scipy의 TruncatedSVD를 이용해 행렬 분해
# sklearn과 차이점 : scipy의 TruncatedSVD는 U, sigma, Vt값을 반환해주는 반면 sklearn은 반환해주지 않음
U, sigma, Vt = svds(user_movie_rating_matrix_scaled, k=12)

In [48]:
print(U.shape)
print(sigma.shape)
print(Vt.shape)

(671, 12)
(12,)
(12, 9064)


In [51]:
# sigma를 대각행렬로 변경
sigma = np.diag(sigma)

In [54]:
# 분해한 행렬을 다시 곱한 후 유저 평균 평점 값을 더해 줌
svd_user_predicted_ratings = np.dot(np.dot(U, sigma),Vt) + user_rating_mean.reshape(-1,1)

In [58]:
svd_user_predicted_ratings_df = pd.DataFrame(svd_user_predicted_ratings, index=user_movie_rating.index, columns=user_movie_rating.columns)

In [60]:
svd_user_predicted_ratings_df

title,"""Great Performances"" Cats (1998)",$9.99 (2008),'Hellboy': The Seeds of Creation (2004),'Neath the Arizona Skies (1934),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...And God Spoke (1993),...And Justice for All (1979),1-900 (06) (1994),10 (1979),10 Attitudes (2001),10 Cloverfield Lane (2016),10 Items or Less (2006),10 Things I Hate About You (1999),10 Years (2011),"10,000 BC (2008)",100 Girls (2000),100 Rifles (1969),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),101 Reykjavik (101 Reykjavík) (2000),102 Dalmatians (2000),"10th Kingdom, The (2000)","11'09""01 - September 11 (2002)",11:14 (2003),12 Angry Men (1957),12 Angry Men (1997),12 Years a Slave (2013),127 Hours (2010),12:08 East of Bucharest (A fost sau n-a fost?) (2006),13 Assassins (Jûsan-nin no shikaku) (2010),13 Ghosts (1960),13 Going on 30 (2004),13 Tzameti (2005),"13th Warrior, The (1999)",...,Zach Galifianakis: Live at the Purple Onion (2006),Zack and Miri Make a Porno (2008),Zardoz (1974),Zathura (2005),Zatoichi on the Road (Zatôichi kenka-tabi) (Zatôichi 5) (1963),Zazie dans le métro (1960),"Zed & Two Noughts, A (1985)",Zeitgeist: The Movie (2007),Zelary (2003),Zelig (1983),Zenon: Girl of the 21st Century (1999),Zenon: The Zequel (2001),Zenon: Z3 (2004),Zero Dark Thirty (2012),Zero Effect (1998),"Zero Theorem, The (2013)",Zerophilia (2005),Zodiac (2007),Zombeavers (2014),Zombie (a.k.a. Zombie 2: The Dead Are Among Us) (Zombi 2) (1979),Zombie Holocaust (a.k.a. Doctor Butcher M.D.) (Zombi Holocaust) (1980),Zombieland (2009),Zoolander (2001),Zoolander 2 (2016),Zoom (2006),Zoot Suit (1981),Zootopia (2016),Zorba the Greek (Alexis Zorbas) (1964),Zorn's Lemma (1970),"Zorro, the Gay Blade (1981)",Zulu (1964),Zulu (2013),[REC] (2007),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931),İtirazım Var (2014)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,-0.004829,0.001440,-0.004414,-0.004888,-0.003999,-0.004063,-0.000314,0.020666,0.004286,0.001170,0.012586,-0.005268,0.039120,-0.007238,0.003286,-0.006070,0.013154,-0.003984,0.023310,-0.002686,0.000853,-0.005667,-0.004888,0.017293,0.058342,-0.003776,-0.007989,-0.005192,-0.004249,-0.006012,0.101904,-0.006973,-0.017342,0.010629,0.005848,0.018616,-0.004644,-0.002115,-0.003144,0.030339,...,-0.003784,0.021700,-0.003087,0.004107,-0.007047,0.000054,0.008282,-0.004066,-0.006185,0.004086,-0.005833,-0.005833,-0.005833,0.019202,0.017190,-0.004573,-0.006070,0.045869,-0.003942,0.017243,-0.004847,0.027236,0.020112,-0.005182,-0.002683,-0.004727,-0.009548,0.004639,0.008914,0.000460,-0.001234,-0.003942,-0.002428,0.024946,-0.006701,0.015451,-0.004710,0.057999,-0.004255,-0.006070
2,0.001307,0.013514,-0.001728,-0.000127,0.021747,-0.002692,-0.021480,-0.018644,-0.009828,0.111083,-0.010260,0.000117,-0.013825,-0.010881,-0.006525,-0.001141,0.025100,-0.009207,0.088380,-0.017234,-0.005557,0.015030,-0.000127,0.140625,0.048861,0.004380,0.005970,-0.001554,-0.002320,0.029402,0.185064,0.015676,0.016771,0.070782,0.003389,0.006995,-0.000954,0.021837,0.013075,-0.043766,...,0.003448,0.006980,-0.011933,-0.019173,-0.015106,0.007021,-0.007787,-0.008222,0.011692,0.014353,-0.001002,-0.001002,-0.001002,0.018789,-0.102010,-0.017634,-0.001141,0.006456,-0.007639,0.054390,-0.000464,-0.032493,-0.024182,0.002100,-0.004119,-0.000641,-0.031981,-0.060058,0.004483,-0.039346,-0.012717,-0.007639,0.007005,-0.124708,0.001648,-0.033993,-0.002859,-0.032061,-0.014109,-0.004992
3,0.011559,0.005199,0.016041,0.011300,-0.002313,0.019623,0.027392,0.008358,0.044102,0.183853,-0.040184,0.013816,0.020495,0.007953,-0.051213,0.008751,-0.044949,-0.006587,0.073125,-0.028985,-0.003561,0.020940,0.011300,-0.021577,0.009370,0.016740,0.036675,0.012658,0.011982,0.032655,0.069909,0.040781

In [119]:
# 개인화 추천 함수
def movie_recommend_by_user(user_id, org_movies_df, predicted_ratings_df, movie_info, num_recommend):
  """
  input : userid, org_movies_df, predicted_ratings_df, num_recommend
  output : recommended movies by user
  description : user_id별로 본 영화를 제외하고 예측결과 순으로 영화 추천
  """

  # predicted_ratings_df에서 해당 user가 안본 영화만 선택
  unseen_movie_index = org_movies_df.transpose()[user_id] == 0
  unseen_movie = predicted_ratings_df.transpose()[user_id][unseen_movie_index].sort_values(ascending=False)

  recommneded_movie_list = pd.DataFrame(unseen_movie[:num_recommend]).reset_index().rename(columns={user_id:'predicted'})
  recommneded_movie_list_with_info = pd.merge(movie_info, recommneded_movie_list, how='right', on='title')

  return recommneded_movie_list_with_info



In [121]:
movie_recommend_by_user(330, user_movie_rating, svd_user_predicted_ratings_df, movies, 10)

,movieId,title,genres,predicted
0,296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,2.059987
1,356,Forrest Gump (1994),Comedy|Drama|Romance|War,1.772624
2,608,Fargo (1996),Comedy|Crime|Drama|Thriller,1.526693
3,590,Dances with Wolves (1990),Adventure|Drama|Western,1.508757
4,380,True Lies (1994),Action|Adventure|Comedy|Romance|Thriller,1.276502
5,47,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,1.265067
6,377,Speed (1994),Action|Romance|Thriller,1.141936
7,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,1.077095
8,357,Four Weddings and a Funeral (1994),Comedy|Romance,1.065929
9,592,Batman (1989),Action|Crime|Thriller,1.048694
